<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

## Das Gradientenverfahren

In diesem Notebook geht es erneut um Regressionsprobleme.
Nur, dass wir diesmal nicht die Funktionen zur Linearen Regression aus der Scikit-Learn Bibliothek verwenden, sonder die Lernalgorithmen selber implementieren.

Um die Wirkungsweise der Algorithmen besser bewerten zu können, verwenden wir im Beispiel keine *echten* Daten, sondern wir generieren einen sythetischen Datensatz mit nur einer unabhängigen Variable.

Die generierten Datenpunkte sollen sich anhand einer Geraden orientieren und um diese Gerade mit einer gewissem gewissen, normalverteilten Fehler (*Rauschen*) angeordnet sein.
Wichtig sind hier die beiden Parameter `ziel_theta_0` und `ziel_theta_1`, die den Achsenabschnitt bzw. die Steigung der Geraden angeben.
Unsere Modellfunktion sollte diese beiden Parameter möglichst gut annähern, da Sie die *wirkliche* Verteilung der Daten angeben. 

Wir werden wie folgt vorgehen:
1. Datensatz erzeugen
2. Python-Funktionen für Modell- und Kostenfunktion entwickeln
3. Kostenfunktion visualisieren
4. Gradientenverfahren implementieren
5. Parameter für die Modellfunktion optimieren

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

size = 1000
mux = 0
stdx = 1
X = np.random.normal(mux,stdx,size)
shiftx = np.min(X)
X = X.reshape(size,1)

munoise = 0
stdnoise = .5
noise = np.random.normal(munoise,stdnoise,size)
noise = noise.reshape(size,1)

ziel_theta_0 = 12
ziel_theta_1 = 4.4

y = ziel_theta_0 + ziel_theta_1*X + noise
print(np.shape(y))

#plt.hist(noise, bins=100)
plt.scatter(X,y)
plt.show()

Um den Bias-Parameter mitverarbeiten zu können, fügen wir eine Spalte zur Datenmatrix hinzu. Jedem Datenpunkt wird dazu ein neues $x_0 = 1$ zugeordnet.

In [ ]:
X_new = np.c_[np.ones((size, 1)), X]

Unsere Modellfunktion $h_{\Theta}(X)$ definieren wir als Python Funktion `h(X,theta)`

In [ ]:
def h(X,theta):
    '''Modellfunktion h
        X ist eine Matrix. Jede Zeile entspricht einem Datenpunkt.    
    '''
    return X@theta

Die Kostenfunktion $J(\Theta)$ definieren wir als Funktion `J_mse(X, y, theta)`

In [ ]:
def J_mse(X, y, theta):
    # Aus theat einen Numpy Spaltenvektor machen
    theta = np.array(theta).reshape(-1,1)
    # Kosten berechnen
    me = h(X,theta)-y
    r = me.T@me / (2*len(X))
    # Kosten als Skalar zurückgeben
    return r.item(0)

Wir können nun die Kosten für beliebige Schätzungen von $\Theta$ berechnen.

In [ ]:
J_mse(X_new, y, [0.0,0.0])

In der folgenden Code-Zelle visualisieren wir die Kosten als 3D-Graph.
Die $x$ und $y$ Achse geben die verschieden Werte für die Parameter $\Theta_0$ bzw. $\Theta_1$ an, auf der $z$-Achese sind die entsprechenden Kosten aufgetragen.

Die Funktion ist wie eine *Schüssel* geformt, mathematisch gesehen würde man sagen, sie ist *konvex*.
Bei dieser Funktion wird das Gradientfahren garantiert das (globale) Optimum finden.
Die Wahl der Parameter ist optimal am tiefsten Punkt der "Schüssel".
Der Gradientabstieg wird sich, unter gewissen Bedingungen, in jedem Schritt diesem Optimum nähern.
Bei jeder Wahl der Parameter $\Theta_0$ und $\Theta_1$ wird berechnet, in welcher Richtung der steilste Abstieg hin zum Optimum erfolgen muss.
In dieser Richtung wählt man einen neue Parameter-Satz aus und berechnet die Kosten an diesem Punkt, berechnet erneut den Gradienten, usw.

In [ ]:
%matplotlib inline
t0 = np.linspace(-50, 75, 30)
t1 = np.linspace(-50, 75, 30)

xx, yy = np.meshgrid(t0, t1)
xy = list(zip(np.reshape(xx,-1),np.reshape(yy,-1)))
zz = np.array([J_mse(X_new, y, np.array(i).reshape(-1,1)) for i in xy]).reshape(np.shape(xx))

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(xx, yy, zz, cmap=cm.coolwarm)

ax.set_xlabel(r'$\theta_0$')
ax.set_ylabel(r'$\theta_1$')
ax.set_zlabel(r'J($\theta$)=MSE($\theta$)')


plt.savefig("Kostenfunktion3D.png",transparent=True, dpi=300)
plt.show()

In der Ebene kann man diese dreidimensionale Ansicht als Konturdiagramm darstellen.
Diese Art von Diagramm ist eine Art Draufsicht auf die Funktion entlang der $z$-Achse.
Die $z$-Werte werden dabei farblich kodiert dargestellt.
Sie kennen diese Darstellung vermutlich als *Höhenlinien* aus geografischen Landkarten.

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.contour(xx, yy, zz, 10, extend='both') #, vmin=0, vmax=1000000)
plt.xlabel(r'$\theta_0$')
plt.ylabel(r'$\theta_1$')
plt.show()

In der folgenden Zelle definieren wir das Gradientenverfahren als Python-Funktion `gradient_descent`.
Die Funktion bekommt als als Parameter u.a. die Anzahl von Iterationen übergeben, nach der das Verfahren stoppen soll.
Es gibt für das Gradientenverfahren auch andere, evtl. sogar bessere Abbruchkriterien.

In [ ]:
def gradient_descent(X, y, theta, alpha, iterationen):
    kosten = []
    # Aus theat einen Numpy Spaltenvektor machen
    theta = np.array(theta).reshape(-1,1)
    for iter in range(iterationen):
        kosten.append(J_mse(X, y, theta))
        gradient = 1/len(y) * (X.T @ (X @ theta - y))
        theta = theta - (alpha * gradient)
    return theta, kosten

Wir wollen nun das Verfahren ausprobieren und das oben definierte Modell minimieren.
In der folgenden Code-Zelle werden wir die Funktion `gradient_descent` mehrfach mit verschiedenen Lernraten zwischen $0.03$ und $2.0$ ausprobieren.

Wir sehen, dass die verschiedenen Lernraten mehr oder weniger schnell zum Ziel führen.
Bei der Lernrate $2.0$ scheint das Verfahren sogar zu divergieren, also in die *falsche Richtung* zu laufen.
Dies ist ein Zeichen dafür, dass die Lernrate zu hoch gewäht ist, und das Gradientenverfahren in zu großen Schritten, quasi *über das Optimum hinwegspringt*.
Kleinere Lernraten füren sicher zum Ziel, haben aber den Nachteil, dass sehr viele Schritte benötigt werden.

Die Wahl der Lernrate ist allgemein ein kritischer Faktor für die Qualität eines Optimierungsalgorithmus, wie *Gradient Descent*.
Viele Verfahren bestimmen die Lernrate aufgrund der einzelnen Optimierungsschritte.

In [ ]:
theta_0 = [0,0]

laeufe = []
alphas = [0.03, 0.1, .4, 1.8]
for alpha in alphas:
    theta_gd, kosten = gradient_descent(X_new, y, theta_0, alpha, 30)
    laeufe.append(kosten)
    
plt.rcParams.update({'font.size': 14})

fig = plt.figure(figsize=(10,6))
for i in range(0,len(laeufe)):
    plt.scatter(range(0,len(laeufe[i])),laeufe[i][0:], marker="x", label=r'$\alpha=$'+str(alphas[i]))

plt.legend(loc='upper center', ncol=2)
plt.ylim([-20,120])
plt.ylabel(r'$J(\Theta)$')
plt.xlabel(r'Epoche')
plt.savefig("lernrate.png",transparent=True, dpi=300)
plt.show()

Die Abbildung in der nächsten Code-Zelle Zeigt, wie nah die durch das Gradientenverfahren bestimmte Lösung am tatsächlichen Optimum liegt.
Beachten Sie die Wertebereiche der Parameter im Konturdiagramm.
Die *Höhenlinien* umfassen nur sehr wenige Zehntel und die beiden Lösungen liegen selbst dabei sehr dicht zusammen.

In [ ]:
fig = plt.figure(figsize=(12,8))

theta_gd, kosten = gradient_descent(X_new, y, theta_0, 0.3, 20)
theta_opt = np.linalg.inv(X_new.T @ X_new) @ X_new.T @ y

scope = .5

t0 = np.linspace(theta_gd[0]-scope, theta_gd[0]+scope, 30)
t1 = np.linspace(theta_gd[1]-scope, theta_gd[1]+scope, 30)

xx, yy = np.meshgrid(t0, t1)
xy = list(zip(np.reshape(xx,-1),np.reshape(yy,-1)))
zz = np.array([J_mse(X_new, y, np.array(i).reshape(-1,1)) for i in xy]).reshape(np.shape(xx))

plt.contour(xx, yy, zz, 10, extend='both') #, vmin=0, vmax=1000000)
plt.xlabel('Theta_0')
plt.ylabel('Theta_1')
theta_gd_plt = theta_gd.T[0]
theta_opt_plt = theta_opt.T[0]
plt.scatter(theta_gd_plt[0],theta_gd_plt[1],c="r")
print(theta_gd)
print(theta_opt)
plt.xlabel(r'$\theta_0$')
plt.ylabel(r'$\theta_1$')
plt.annotate('Optimum (Normalgleichung)', xy=theta_opt_plt, xytext=(theta_opt_plt[0]-scope/2, theta_opt_plt[1]+scope/2),
            arrowprops=dict(facecolor='black', width=.5, headwidth=5))
plt.annotate('Approximiert (Gradientenverfahren)', xy=theta_gd_plt, xytext=(theta_gd_plt[0]-scope/2, theta_gd_plt[1]-scope/2),
            arrowprops=dict(facecolor='black', width=.5, headwidth=5))

plt.show()

In der nächsten Code-Zelle wollen wir darstellen, wie sich das Gradientenverfahren in seinen einzelenen Iterationen Schritt für Schritt dem Optimum nähert.
Wir sehen, dass alle Schritte die Parameter auf direktem Weg zum Optimum bewegen.

Die Code-Zelle erzeugt übrigens auch eine GIF Grafik, die Sie sich in einem Anzeigeprogramm ansehen können.
Darin sind die Schritte im zeitlichen Verlauf simuliert.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
plt.rcParams.update({'font.size': 22})

scope = 2

t0 = np.linspace(theta_gd[0]-scope, theta_gd[0]+scope, 30)
t1 = np.linspace(theta_gd[1]-scope, theta_gd[1]+scope, 30)

fig, ax = plt.subplots(figsize=(12,8))
plt.contour(xx, yy, zz, 10, extend='both') #, vmin=0, vmax=1000000)
plt.xlabel(r'$\theta_0$')
plt.ylabel(r'$\theta_1$')
limx = ax.get_xlim()
limy = ax.get_ylim()

#Probiere verschiedene Werte für alpha:
alpha=.3
iterationen = 30
theta_0 = np.array([theta_gd[0]+scope, theta_gd[1]+scope]).reshape(2,1)

p0 = []
p1 = []
kosten = []

def gd_plot(i):
    global theta_0
    global kosten
    v0 = theta_0.item(0)
    v1 = theta_0.item(1)
    p0.append(v0)
    p1.append(v1)
    ax.plot(p0, p1, "-x", color='r')
    ax.set_xlim(limx)
    ax.set_ylim(limy)
    kosten.append(J_mse(X_new, y, theta_0))
    gradient = 1/len(y) * (X_new.T @ (X_new @ theta_0 - y))
    theta_0 = theta_0 - (alpha * gradient)
    if(i==iterationen-1):
        ax.scatter(theta_0.item(0), theta_0.item(1), marker="x", c="b", s=100, zorder=i+10)
        plt.savefig("lernrate_GD_contour.png",transparent=True, dpi=300)
    return


anim = FuncAnimation(fig, gd_plot, frames=np.arange(0, iterationen), interval=100, repeat=True)
anim.save('gd_contour_80.gif', dpi=80, writer='pillow')

![](gd_contour_80.gif)

Auch in der Abbildung der Fehlerrate sehen wir, das jeder Schritt die Wahl der Parameter verbessert.

In [ ]:
plt.plot(range(0,len(kosten)),kosten[0:], "x-", label=str(alphas[i]))
plt.savefig("lernrate_GD.png",transparent=True, dpi=300)

In der folgenden Code-Zelle verwenden wir nun eine Variante des Gratienverfahrens, den sogenannten **Mini-Batch Gradientenabstieg**.
Beim Mini-Batch Verfahren werden für die Neuberechnung der Parameter nicht alle, sondern nur ein gewisser Teil von $k$ Datenpunkten ausgewertet.
Die Konstante $k$ wird *batch*-Größe genannt und ist im Normalfall deutlich kleiner als die Anzahl der Datenpunkte $n$ im Trainigsdatensatz.

Wir verwenden hier eine *batch*-Größe von 64 Datenpunkten, die außerdem jeweils zufällig auswählen.
Statt die Kosten für alle 1000 Punkte zu berechnen, bestimmen wir also nur die Kosten für 64 zufällige Datenpunkte.
Danach berechnen wir den Gradienten, verschieben die Parameter-Werte in Richtung dieses Gradienten und wählen erneut 64 zufällige Punkte für den nächsten Schritt aus.

Wir sehen, dass sich die Wahl der Parameter beim Mini-Batch Gradientenabstieg nicht so stringent dem Minimum nähert, wie beim vollständigen Verfahren.
Allerdings kommen wir deutlich schneller zum Ziel, da die Parameter bereits nach der Berechnung von wenigen Datenpunkten korrigiert werden.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


fig, ax = plt.subplots(figsize=(12,8))
plt.contour(xx, yy, zz, 10, extend='both') #, vmin=0, vmax=1000000)
plt.xlabel(r'$\Theta_0$')
plt.ylabel(r'$\Theta_1$')
limx = ax.get_xlim()
limy = ax.get_ylim()

alpha=.3
iterationen = 30
#theta_0 = np.random.randn(1,2)
theta_0 = np.array([theta_gd[0]+scope, theta_gd[1]+scope]).reshape(2,1)

p0 = []
p1 = []
kosten = []
batchsize = 64

def gd_plot(i):
    global theta_0
    global kosten
    v0 = theta_0.item(0)
    v1 = theta_0.item(1)
    p0.append(v0)
    p1.append(v1)
    ax.plot(p0, p1, "-x", color='r')
    ax.set_xlim(limx)
    ax.set_ylim(limy)
    idx = np.random.randint(X.shape[0], size=batchsize)
    xi = X_new[idx,:]
    yi = y[idx,:]
    kosten.append(J_mse(xi, yi, theta_0))
    gradient = 1/len(yi) * (xi.T @ (xi @ theta_0 - yi))
    theta_0 = theta_0 - (alpha * gradient)
    if(i==iterationen-1):
        ax.scatter(theta_0.item(0), theta_0.item(1), marker="x", c="b", s=400, zorder=i+10)
        plt.savefig("lernrate_SGD_contour.png",transparent=True, dpi=300)
    return

anim = FuncAnimation(fig, gd_plot, frames=np.arange(0, iterationen), interval=200, repeat=True)
anim.save('bgd_contour_80.gif', dpi=80, writer='pillow')

![](bgd_contour_80.gif)

Auch bei den Kosten sehen wir, dass nicht jeder Schritt eine Verbesserung bringt.
Allerdings benötigt der Mini-Batch Gradientenabstieg nur sehr wenige Epochen, um sich dem Optimum recht gut zu nähern.

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
gens = np.linspace(0, len(kosten)/(len(X_new)/batchsize), len(kosten))

plt.plot(gens,kosten[0:], "x-")
plt.ylabel(r'$J_{\Theta}(x)$')
plt.xlabel('Epoche')
plt.show()

## Multiple und polynomiale Regression

Im obigen Beispiel haben wir ein Regressionsproblem mit nur einer unabhängigen Variabel (oder auch *Merkmal*) betrachtet.
Dies ist hilfreich für die Visulaisierung, aber natürlich nicht der Normalfall.
In der Praxis haben unsere Datensätze weit mehr als nur ein Merkmal.
Wir habe dies ja bereits in den vorherigen Beispielen zur linearen Regression gesehen.

In der multiplen Linearen Regression gehen wir davon aus, dass sich die abhängige Variable als Linearkombination der unabhängigen Variablen voraussagen lässt.
Was ist aber, wenn ein solcher linearer Zusammenhang nicht besteht?
Dann kann es sein, dass ein nicht-linearer Zusammenhang besteht und sich die Zielvariable $y$ durch eine Polynomfunktion der unabhängigen Variablen erklären lässt.

Bei einem derartigen Zusammenhang kann uns die sogenannte **polynomiale Regression** helfen.
Die polynomiale Regression ist ein Spezialfall der multiplen linearen Regression, bei der zusätzlich zu den unabhängigen Variablen noch die polynome bis zum Grad $n$ mit in die Merkmalsvariablen aufgenommen werden.
Bei zwei unabhängigen Variablen $x_0$ und $x_1$ herhalten wir für ein Polynom zweiten Grades also die Modellfunktion:\
$\hat{y} = \Theta_0 + \Theta_1 x_0 + \Theta_2 x_1 + \Theta_3 x_0^2 + \Theta_4 x_1^2 + \Theta_5 x_0 x_1$

Im folgenden Beispiel verwenden wir erneut einen generierten Datensatz mit nur einer unabhängigen Variablen $x$.
Unser Modell verwendet ein Polynaom zweiten Grades.

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

fig, ax = plt.subplots(figsize=(10,6))

size = 100
mux = 0
stdx = 0.5
#X = np.random.normal(mux,stdx,size)
X = np.random.uniform(-1, 1, size)
xmin = np.min(X)
xmax = np.max(X)

X = np.c_[X, X*X]

munoise = 0
stdnoise = 0.4
noise = np.random.normal(munoise,stdnoise,size)
noise = noise.reshape(size,1)

ziel_theta_0 = 12
ziel_theta_1 = -4.4
ziel_theta_2 = 28.1

y = ziel_theta_0 + ziel_theta_1*X[:,[0]] + ziel_theta_2*X[:,[1]] + noise

plt.scatter((X[:,[0]]).reshape(size), y.reshape(size))

vone = (np.ones(size)).reshape(size,1)
X = np.concatenate((vone, X), 1)



#theta_0 = np.random.randn(X.shape[1],1)
theta_0 = [0,0,0]
theta, costs = gradient_descent(X, y, theta_0, 0.8, 50)

xi = np.linspace(xmin,xmax,30)
yi = [ theta.item(0) + theta.item(1)*i + theta.item(2)*i*i for i in xi ]
plt.plot(xi,yi, c='red')
plt.show()

theta

## Overfitting

Overfitting ist ein ein generelles Problem im Maschinellen Lernen.
Auch bei der Wahl der Polynomfunktion für eine Polynomiale Regression kann zu Überanpassung kommen.
Wenn man dem Modell so hohe Freiheitsgrade gibt, dass die Funktion sich den Daten sehr genau anpassen kann, verliert das Modell die Fähigkeit zu *Veralgemeinern*.

Das folgende Beispiel zeigt, etwas überspitzt, was passiert, wenn man eine Polynomfunktion an **wenige** Daten anpasst.
Im Beispiel transformieren wir einen Datensatz in einen Datensatz, der die Variablen für das Polynom 5. Grades enthält.
Die *normale* Lineare Regression passt sich den Daten **ideal** an.
Aber offensichtlich ist das entstehende Modell nicht gut, denn es spiegelt nicht den *Verlauf* der Daten wieder.

Setzt man **Regularisierung**, z.B. in Form der **Lasso-** oder **Ridge-Regression** ein, sind die entstehenden Modellfunktionen deutlich sinnvoller.

In [ ]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
%matplotlib inline

fig, ax = plt.subplots(figsize=(10,6))

samples = 300
size = 10
mux = 0
stdx = 0.5
#X = np.random.normal(mux,stdx,size)
X = np.random.uniform(-1, 1, size)
xmin = np.min(X)
xmax = np.max(X)
X = np.c_[X, X*X]


xi = np.linspace(xmin,xmax,samples)
xi = np.c_[xi, xi*xi]
vone = (np.ones(samples)).reshape(samples,1)
xi = np.concatenate((vone, xi), 1)

munoise = 0
stdnoise = 1.4
noise = np.random.normal(munoise,stdnoise,size)
noise = noise.reshape(size,1)

ziel_theta_0 = 12
ziel_theta_1 = -4.4
ziel_theta_2 = 28.1

y = ziel_theta_0 + ziel_theta_1*X[:,[0]] + ziel_theta_2*X[:,[1]] + noise

plt.scatter((X[:,[0]]).reshape(size), y.reshape(size))

vone = (np.ones(size)).reshape(size,1)
X = np.concatenate((vone, X), 1)
    

##model = LinearRegression()
##model.fit(X, y)
poly_reg=PolynomialFeatures(degree=5)
X_poly=poly_reg.fit_transform(X)
poly_reg.fit(X_poly,y)
model=LinearRegression()
model.fit(X_poly,y)
xi_poly = poly_reg.fit_transform(xi)
ys = model.predict(xi_poly)



model = Ridge(alpha=0.5)
model.fit(X_poly, y)
yr = model.predict(xi_poly)

model = Lasso(alpha=0.5)
model.fit(X_poly, y)
yl = model.predict(xi_poly)


plt.plot(xi[:,[1]],ys, label="Standard", c='red')
plt.plot(xi[:,[1]],yr, label="Ridge")
plt.plot(xi[:,[1]],yl, label="Lasso")
#plt.ylim(np.min(y)-100,np.max(y)+100)
plt.legend()
plt.savefig("regularisierung.png",transparent=True, dpi=300)
plt.show()